# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 2

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        'NYC_summer_2010_99p',
        # 'NYC_summer_2015_50p',
        # 'NYC_summer_2017_25p',
        # 'NYC_summer_2018_75p'
    ]),
    # ('PHX_Heat_Test', [
    #     'PHX_summer_2008_25p',
    #     'PHX_summer_2009_50p',
    #     'PHX_summer_2011_99p',
    #     'PHX_summer_2015_75p',
    #     'PHX_summer_2020_01p'
    # ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

print(sim_names)
client = storage.Client(project="climateiq")


2024-12-13 20:12:24.968461: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 20:12:25.019090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 20:12:25.019128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 20:12:25.020434: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 20:12:25.028482: I tensorflow/core/platform/cpu_feature_guar

['NYC_Heat_Test/NYC_summer_2000_01p', 'NYC_Heat_Test/NYC_summer_2010_99p']


In [2]:
# Create the training dataset using create_atmo_dataset with sim_names
train_frac = 0.8

train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(0.0, train_frac),
).batch(batch_size=batch_size)

val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(train_frac, 1.0),
).batch(batch_size=batch_size)

INFO:root:sim_name_dates [('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-07'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-22'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-07-13'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-07-16'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-06-18'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-08-31'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-08-02'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-09'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-07-03'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-10'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-06-13'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-07-12'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-13'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-08-10'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-07-01'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-24'), ('NYC_Heat_Test/NYC_summer_2000_01p', '2000-06-11'), ('NYC_Heat_Test/NYC_summer_2010_99p', '2010-05-30'), ('NYC_Heat_Test/NYC_

In [3]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)

In [4]:
# Train the model
tb_callback = keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(train_ds, val_ds, epochs=1, callbacks=[tb_callback])

Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)
Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)


2024-12-13 20:12:46.537737: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm/conv_lstm/conv_lstm2d/while/body/_1/atmo_conv_lstm/conv_lstm/conv_lstm2d/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Output shape: TensorShape([None, 8, 50, 50, 256])


INFO:root:load_day (2010-06-18, NYC_Heat_Test/NYC_summer_2010_99p)
INFO:root:load_day (2000-05-24, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2000-08-14, NYC_Heat_Test/NYC_summer_2000_01p)
2024-12-13 20:12:51.752692: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900
INFO:root:load_day (2000-07-21, NYC_Heat_Test/NYC_summer_2000_01p)
INFO:root:load_day (2010-06-28, NYC_Heat_Test/NYC_summer_2010_99p)


      1/Unknown - 15s 15s/step - loss: 18248.9062 - mean_absolute_error: 74.0046 - root_mean_squared_error: 135.0885Output shape: TensorShape([None, 8, 50, 50, 256])


...

Output shape: TensorShape([None, 8, 50, 50, 256])


INFO:root:load_day (2010-07-26, NYC_Heat_Test/NYC_summer_2010_99p)


Output shape: TensorShape([None, 8, 50, 50, 256])


2024-12-13 20:18:49.331424: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14741129010360194129
2024-12-13 20:18:49.331468: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12283461792789415023
2024-12-13 20:18:49.331477: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4826752829124410403
2024-12-13 20:18:49.331483: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15402361440212274947
2024-12-13 20:18:49.331504: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13682838528160178272


15/15 [==============================] - 367s 25s/step - loss: 18424.3574 - mean_absolute_error: 74.0701 - root_mean_squared_error: 135.7364 - val_loss: 18029.3652 - val_mean_absolute_error: 73.2409 - val_root_mean_squared_error: 134.2735


In [5]:
model._model.summary()

Model: "atmo_conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  488       
                                                                 
 spatial_cnn (Sequential)    (None, 50, 50, 128)       252992    
                                                                 
 spatiotemporal_cnn (Sequen  (None, None, 50, 50, 64   30480     
 tial)                       )                                   
                                                                 
 conv_lstm (Sequential)      (None, None, 50, 50, 51   45877248  
                             2)                                  
                                                                 
 t2_output_cnn (Sequential)  (None, None, 200, 200,    69729     
                             1)                                  
                                                    

In [6]:
# Test calling the model on some training data
inputs, labels = next(iter(train_dataset))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)

NameError: name 'train_dataset' is not defined